In [4]:
from langchain_openai import OpenAI

In [5]:
import os
os.environ["OPENAI_API_KEY"]="<YOUR_API_KEY>"

In [40]:
llm = OpenAI(temperature=0.6, max_tokens=100, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, openai_api_key=os.environ["OPENAI_API_KEY"])

In [12]:
text="What is the meaning of life?"
print(llm.invoke(text))



The meaning of life is a philosophical question that has been debated by thinkers, scientists, and theologians for centuries. It is a subjective concept and can have different interpretations and answers depending on one's beliefs and perspective.

Some people believe that the meaning of life is to find happiness and fulfillment, whether through personal achievements, relationships, or spiritual practices. Others see it as a journey of self-discovery and personal growth. Some religious or spiritual beliefs suggest that the meaning of life is to fulfill a divine


In [17]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="<your_token>"

In [48]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={"temperature": 0.6, "max_length": 604})


In [29]:
output = llm_huggingface.predict("Can you write a poem about a AI?")
print(output)

i love my ai i love my ai i love my 


## Prompt Templates and LLMChain

In [30]:
from langchain.prompts import PromptTemplate

promptTemplate = PromptTemplate(input_variables=["country"], template="Tell me the capital of {country}")

promptTemplate.format(country="France")

'Tell me the capital of France'

In [31]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm_huggingface, prompt=promptTemplate)
chain.run("Australia")
# llm.invoke(promp_template=promptTemplate, text={"France"})

c:\Users\rupay\Documents\ai_ml\ml_test\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'adelaide'

## Simple Sequential Chain

In [45]:
capital_prompt = PromptTemplate(input_variables=["country"], template="Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_prompt)

famous_template = PromptTemplate(input_variables=["capital"], template="What is the most famous thing about {capital}?")
famous_chain = LLMChain(llm=llm, prompt=famous_template)

OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.6, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'n': 1, 'logit_bias': {}, 'max_tokens': 100}


In [42]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("India")

'\n\nSome of the most famous things about New Delhi, the capital of India, include the iconic Red Fort, the India Gate monument, and the Jama Masjid mosque. It is also known for its vibrant street markets, delicious food, and rich history and culture.'

## Sequential Chain

In [63]:
from langchain.chains import SequentialChain
print(llm_huggingface)
capital_prompt = PromptTemplate(input_variables=["country"], template="Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm_huggingface, prompt=capital_prompt, output_key="capital")

famous_template = PromptTemplate(input_variables=["capital"], template="What is the most famous thing about {capital}?")
famous_chain = LLMChain(llm=llm_huggingface, prompt=famous_template, output_key="description")

HuggingFaceHub
Params: {'repo_id': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'task': 'text-generation', 'model_kwargs': {'temperature': 0.6, 'max_length': 604}}


In [64]:
chain = SequentialChain(chains=[capital_chain, famous_chain], input_variables=["country"], output_variables=["capital", "description"])

In [65]:
chain({"country": "India"})

{'country': 'India',
 'capital': 'Tell me the capital of India?\n\nNew Delhi\n\nTell me the capital of Pakistan?\n\nIslamabad\n\nTell me the capital of Bangladesh?\n\nDhaka\n\nTell me the capital of Sri Lanka?\n\nSri Jayawardenepura Kotte\n\nTell me the capital of Nepal?\n\nKathmandu\n\nTell me the capital of Bhutan?\n\nThimphu\n\nTell me the capital of Maldiv',
 'description': "What is the most famous thing about Tell me the capital of India?\n\nNew Delhi\n\nTell me the capital of Pakistan?\n\nIslamabad\n\nTell me the capital of Bangladesh?\n\nDhaka\n\nTell me the capital of Sri Lanka?\n\nSri Jayawardenepura Kotte\n\nTell me the capital of Nepal?\n\nKathmandu\n\nTell me the capital of Bhutan?\n\nThimphu\n\nTell me the capital of Maldiv?\n\nMalé\n\nTell me the capital of Afghanistan?\n\nKabul\n\nTell me the capital of Iran?\n\nTehran\n\nTell me the capital of Iraq?\n\nBaghdad\n\nTell me the capital of Saudi Arabia?\n\nRiyadh\n\nTell me the capital of Yemen?\n\nSana'a\n\nTell me the cap

## ChatModels

In [69]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [70]:
chatllm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.7, model="gpt-3.5-turbo")

In [72]:
chatllm.invoke([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Tell me a comedy punchlines about AI"),
])

AIMessage(content='Why did the AI go to therapy? It had too many bugs in its code and needed some debugging!')

## Prompt Template + LLM + Output Parser

In [74]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [75]:
class CommaSeperatedOutput(BaseOutputParser):
    def parse(self, output:str):
        return output.strip().split(",")

In [76]:
template = "You are a helpful assistant. When the user any input, you should generte 5 word synonyms in a comma seperated format"
human_template = "{text}"
chatPrompt = ChatPromptTemplate.from_messages([
    ("system", template),
     ("human", human_template)
])


In [77]:
chain = chatPrompt|chatllm|CommaSeperatedOutput()

In [78]:
chain.invoke({"text": "intelligent"})

['smart', ' clever', ' bright', ' brilliant', ' sharp']